In [1]:
pwd

'/home/wsuser/work'

In [2]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

In [3]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='eqRr_BuHHp9Di6TVmW-MGXHyPXtsbYlsKBxJDrYcV7Yn',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'naturaldisastersintensityanalysis-donotdelete-pr-wij7ez8jjgzubp'
object_key = 'dataset.zip'

streaming_body_2 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [4]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_2.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [5]:
data_path = '/home/wsuser/work/Dataset/'
batch_size = 32
target_size = (64, 64)

In [6]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   horizontal_flip=True,
                                   validation_split=0.2)

test_datagen = ImageDataGenerator(rescale=1. / 255, validation_split=0.2)

In [8]:
X_train = train_datagen.flow_from_directory('/home/wsuser/work/dataset/train_set',
                                            target_size=(64, 64),
                                            batch_size=32,
                                            color_mode="rgb",
                                            subset="training",
                                            class_mode='categorical')



X_test = test_datagen.flow_from_directory('/home/wsuser/work/dataset/test_set',
                                          target_size=(64, 64),
                                          batch_size=32,
                                          color_mode="rgb",
                                          subset="validation",
                                          class_mode='categorical')

Found 595 images belonging to 4 classes.
Found 37 images belonging to 4 classes.


In [9]:
model = Sequential()

In [10]:
model.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=4, activation='softmax'))

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 128)               8

In [12]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
model.fit(X_train, 
          steps_per_epoch=len(X_train), 
          epochs=20,
          validation_data=X_test,
          validation_steps=len(X_test))

Epoch 1/20
19/19 [==============================] - 28s 1s/step - loss: 1.2825 - accuracy: 0.4017 - val_loss: 1.1511 - val_accuracy: 0.5946
Epoch 2/20
19/19 [==============================] - 25s 1s/step - loss: 1.0187 - accuracy: 0.5933 - val_loss: 0.9829 - val_accuracy: 0.6757
Epoch 3/20
19/19 [==============================] - 25s 1s/step - loss: 0.8670 - accuracy: 0.6571 - val_loss: 1.2125 - val_accuracy: 0.4865
Epoch 4/20
19/19 [==============================] - 26s 1s/step - loss: 0.8239 - accuracy: 0.6723 - val_loss: 0.9556 - val_accuracy: 0.6757
Epoch 5/20
19/19 [==============================] - 25s 1s/step - loss: 0.7818 - accuracy: 0.6840 - val_loss: 1.0185 - val_accuracy: 0.6757
Epoch 6/20
19/19 [==============================] - 26s 1s/step - loss: 0.6352 - accuracy: 0.7714 - val_loss: 0.9754 - val_accuracy: 0.7568
Epoch 7/20
19/19 [==============================] - 26s 1s/step - loss: 0.5867 - accuracy: 0.7681 - val_loss: 0.8624 - val_accuracy: 0.7568
Epoch 8/20
19/19 [==

In [14]:
model.save("disaster.h5")

In [15]:
!tar -zcvf disaster.tgz disaster.h5

disaster.h5


In [16]:
ls

dataset/  disaster.h5  disaster.tgz


In [17]:
!pip install watson-machine-learning-client

     |████████████████████████████████| 538 kB 12.9 MB/s eta 0:00:01


In [18]:
from ibm_watson_machine_learning import APIClient

API_KEY = "dI8836cei5gsTnRg5IXx2Jla_HxwNj1-FyYdDTq1oHkV"

wml_credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": API_KEY
}

client = APIClient(wml_credentials)

In [19]:
client=APIClient(wml_credentials)

In [20]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']['name'] == space_name)['metadata']['id'])

In [21]:
space_uid = guid_from_space_name(client, 'Natural Disasters Intensity Analysis')
print("Space UID: ", space_uid)

Space UID:  eaa2304c-ca2e-4e95-a4a0-23809b39bdda


In [22]:
client.set.default_space(space_uid)

'SUCCESS'

In [23]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [24]:
software_spec_uid = client.software_specifications.get_uid_by_name("runtime-22.1-py3.9")
software_spec_uid

'12b83a17-24d8-5082-900f-0ab31fbfd3cb'

In [25]:
model_details = client.repository.store_model(model="disaster.tgz", meta_props={
    client.repository.ModelMetaNames.NAME: "CNN",
    client.repository.ModelMetaNames.TYPE: "tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid
})

model_id = client.repository.get_model_id(model_details)

In [26]:
model_id

'19b34bdf-1aaa-4638-97e9-b2fc0d3231cf'

In [27]:
client.repository.download(model_id, "my_model.tar.gz")

Successfully saved model content to file: 'my_model.tar.gz'


'/home/wsuser/work/my_model.tar.gz'